In [2]:
import os
%pwd

'/home/towet/Desktop/Visions/OpenProjects/Malaria-Cell-Classification/research'

In [3]:
os.chdir('../')

In [14]:
%pwd

'/home/towet/Desktop/Visions/OpenProjects/Malaria-Cell-Classification'

In [15]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path 
    source_URL: Path 
    local_data_file: Path 
    unzip_dir: Path
    splited_data: Path

In [6]:
from MalariaClassifier.constants import *
from MalariaClassifier.utils.common import read_yaml, create_directories

In [16]:
class ConfigurationManager:
    def __init__(self, 
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir, config.splited_data])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
            splited_data=config.splited_data
        )
        return data_ingestion_config

In [8]:
import gdown
import zipfile
from MalariaClassifier import logger
import shutil
import splitfolders

In [18]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

    def split_dataset(self):
        splitfolders.ratio(self.config.unzip_dir,
                           self.config.splited_data,
                           seed=1337, ratio=(.7, .2, .1), group_prefix=None,
                           move=False)


    

In [19]:
try:
    config = ConfigurationManager()
    data_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    data_ingestion.split_dataset()
except Exception as e:
    raise e

[2024-01-26 14:20:45,922: INFO: common: yaml file: config/config.yaml loaded successfully]


[2024-01-26 14:20:46,020: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-26 14:20:46,025: INFO: common: created directory at: artifacts]
[2024-01-26 14:20:46,027: INFO: common: created directory at: artifacts/data_ingestion]
[2024-01-26 14:20:46,030: INFO: common: created directory at: artifacts/data_ingestion/dataset]
[2024-01-26 14:20:46,031: INFO: 1300457089: Downloading data from https://drive.google.com/file/d/1fe6kkNHjRp00sBVqmosUfnQ7-Ox1NxNd/view?usp=sharing into file artifacts/data_ingestion/cell-images-for-detecting-malaria.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1fe6kkNHjRp00sBVqmosUfnQ7-Ox1NxNd
From (redirected): https://drive.google.com/uc?/export=download&id=1fe6kkNHjRp00sBVqmosUfnQ7-Ox1NxNd&confirm=t&uuid=2700ed30-c2ed-4ec0-8205-fc8994b9f0ef
To: /home/towet/Desktop/Visions/OpenProjects/Malaria-Cell-Classification/artifacts/data_ingestion/cell-images-for-detecting-malaria.zip
100%|██████████| 352M/352M [05:34<00:00, 1.05MB/s] 

[2024-01-26 14:26:22,687: INFO: 1300457089: Downloaded data from https://drive.google.com/file/d/1fe6kkNHjRp00sBVqmosUfnQ7-Ox1NxNd/view?usp=sharing into file artifacts/data_ingestion/cell-images-for-detecting-malaria.zip]



Copying files: 27560 files [02:59, 153.52 files/s]
